LOGISTIC REGRESSION

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Preparing the data

In [3]:
bc  = datasets.load_breast_cancer()  #binary classification problem predict cancer or not based on input features
X,y = bc.data,bc.target

n_samples,n_features = X.shape
print(n_samples,n_features)

#Train-test split
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size = 0.2, random_state = 1234)

#scale the features
sc = StandardScaler() #helps our feature to have 0 mean and unit variance (always recommended todo when we deal with logistic regression)
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

#now convert our data to torch tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

#Reshaping y_train and y_test
y_train = y_train.view(y_train.shape[0],1)
y_test = y_test.view(y_test.shape[0],1)

569 30


Model

In [4]:
# f = wx+b, sigmoid at the end

class LogisticRegression(nn.Module):
    
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.Linear = nn.Linear(n_input_features,1)
        
    def forward(self,x):
        y_hat = torch.sigmoid(self.Linear(x))  #sigmoid brings the predicted values inside 0 to 1 range 
        return y_hat
        
model = LogisticRegression(n_features)

Loss and Optimizer

In [5]:
learning_rate = 0.01
criterion = nn.BCELoss()  #binary cross entopy loss
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

Training Loop

In [6]:
num_epochs = 100
for epoch in range(num_epochs):
    
    #forward pass and loss
    y_hat = model(X_train)
    loss = criterion(y_hat,y_train)
    
    #backward pass
    loss.backward()
    
    #update weights
    optimizer.step()
    
    #empty the gradients (because .backward() function will add up all the gradients in .grad attribute over multiple epochs)
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f"epoch: {epoch+1}, loss = {loss.item():.4f} ") 
    

epoch: 10, loss = 0.3431 
epoch: 20, loss = 0.3164 
epoch: 30, loss = 0.2950 
epoch: 40, loss = 0.2773 
epoch: 50, loss = 0.2624 
epoch: 60, loss = 0.2496 
epoch: 70, loss = 0.2386 
epoch: 80, loss = 0.2289 
epoch: 90, loss = 0.2203 
epoch: 100, loss = 0.2126 


Evaluating The Model

In [8]:
with torch.no_grad():
    y_predicted = model(X_test)
    #print(y_predicted)
    y_predicted_cls = y_predicted.round() #lower then 0.5 is class 0 and equal to or higher than 0.5 is class 1
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9123
